<h1> Assignment 6
    

<h1> Name & Z-ID 
<h3> Daniel Bendik
<h3> z1938845
<h3> CSCI 490
<h3> Dr. Maoyuan Sun

<h1> Academic Classes

In [325]:
class Academic:
    def __init__(self, campus_id, firstname, lastname):
        self.campus_id = campus_id
        self.firstname = firstname
        self.lastname = lastname
        self.max_credits = self.MAX_CREDITS
        self.schedule = Schedule()
        
    def add_course(self, course):
        if self.schedule.credits + course.number_of_credits > self.max_credits:
            raise Exception(f"Adding this course exceeds the maximum allowed credits. {self.campus_id} has {self.schedule.credits} credits, this course has {course.number_of_credits} credits, but {self.firstname} can only take {self.max_credits} credits at most." )
        self.schedule.add_course(course)

    def remove_course(self, course):
        self.schedule.remove_course(course)

        
class Student(Academic):
    MAX_CREDITS = 16
    def __init__(self, firstname, lastname, campus_id, level):
        super().__init__(firstname, lastname, campus_id)
        self.level = level
        
    def add_course(self, course):
        super().add_course(course)
        course.enroll(self)

    def remove_course(self, course):
        super().remove_course(course)
        course.drop(self)

        
class GraduateStudent(Student):
    MAX_CREDITS = 12
    def __init__(self, firstname, lastname, campus_id):
        super().__init__(firstname, lastname, campus_id, "Graduate")

        
class Instructor(Academic):
    MAX_CREDITS = 9
    def __init__(self, firstname, lastname, campus_id, rank):
        super().__init__(firstname, lastname, campus_id)
        self.rank = rank
        
    def add_course(self, course):
        super().add_course(course)
        course.instructor = self

    def remove_course(self, course):
        super().remove_course(course)
        course.instructor = None

        
s1 = Student("z143", "Catherine", "Smith", "Senior")
s2 = Student("z352", "Niraj", "Kumar", "Sophomore")
s3 = GraduateStudent("z785", "Divya", "Bharti")
s4 = GraduateStudent("z982", "James", "O'Brien")

i1 = Instructor("a421", "Jennifer", "Martinez", "Professor")
i2 = Instructor("a572", "Jonathan", "Jones", "Instructor")

<h1> Course Class

In [326]:
class Course:
    def __init__(self, department, course_number, course_name, section, number_of_credits, times):
        self.department = department
        self.course_number = course_number
        self.course_name = course_name
        self.section = section
        self.number_of_credits = number_of_credits
        self.times = times
        self.instructor = None
        self.students = []

    def __str__(self):
        time_string = ', '.join([f"{day} from {start_time} to {end_time}" for day, start_time, end_time in self.times])
        return f"{self.department} {self.course_number} Section {self.section}: {self.course_name} - {self.number_of_credits} credits - Times: {time_string}"

    def enroll(self, student):
        if student not in self.students:
            self.students.append(student)

    def drop(self, student):
        if student in self.students:
            self.students.remove(student)

    def change_time(self, new_times):
        self.times = new_times
        
    @staticmethod
    def check_time_conflicts(times1, times2):
        for day1, start1, end1 in times1:
            for day2, start2, end2 in times2:
                if day1 == day2 and start1 < end2 and start2 < end1:
                    return True
        return False

c1 = Course("CSCI", 1543, "Programming Principles in Python", 1, 3, [("Mon", 10, 12), ("Wed", 10, 12)])
c2 = Course("CSCI", 1342, "Computer Networks", 2, 4, [("Tue", 14, 16), ("Thu", 14, 16), ("Fri", 12, 13)])
c3 = Course("CSCI", 1352, "Computer Graphics", 1, 3, [("Tue", 10, 12), ("Thu", 10, 12)])
c4 = Course("SOCI", 1230, "Introduction to Sociology", 1, 3, [("Mon", 11, 13), ("Thu", 11, 13)])
c5 = Course("POLS", 1100, "American Politics", 2, 3, [("Tue", 10, 12), ("Thu", 10, 12)])
c6 = Course("SOCI", 1450, "Classical Sociological Theory", 1, 3, [("Mon", 12, 13), ("Wed", 12, 13), ("Fri", 12, 13)])

<h1> Schedule Class

In [327]:
class Schedule:
    def __init__(self, courses=None):
        self.courses = courses if courses is not None else []  # If no courses, make empty list

    @property
    def credits(self):
        return sum(course.number_of_credits for course in self.courses)
            
    def add_course(self, course):
        if course not in self.courses:
            for current_course in self.courses:
                if Course.check_time_conflicts(current_course.times, course.times):
                    raise Exception(f"Time conflict with another course in the schedule. {current_course.course_name} conflicts with {course.course_name}: {current_course.times} and {course.times}")
            self.courses.append(course)

    def remove_course(self, course):
        if course in self.courses:
            self.courses.remove(course)

<h1> Registrar Class

In [328]:
class Registrar:
    def __init__(self):
        self.courses = {}
        self.people = {}

    def add_persons(self, persons):
        for person in persons:
            self.people[person.campus_id] = person

    def add_courses(self, courses):
        for course in courses:
            key = (course.department, course.course_number, course.section)
            self.courses[key] = course

    def add_person_to_course(self, campus_id, department, course_number, section):
        course_key = (department, course_number, section)
        if campus_id not in self.people:
            raise Exception(f"ID {campus_id} not found.")
        if course_key not in self.courses:
            raise Exception(f"{department} {course_number} (section {section}) not found.")
            
        person = self.people[campus_id]
        course = self.courses[course_key]
        person.add_course(course)

    def remove_person_from_course(self, campus_id, department, course_number, section):
        course_key = (department, course_number, section)
        if campus_id not in self.people:
            raise Exception(f"ID {campus_id} not found.")
        if course_key not in self.courses:
            raise Exception(f"{department} {course_number} (section {section}) not found.")
            
        person = self.people[campus_id]
        course = self.courses[course_key]
        person.remove_course(course)
        
    def change_course_time(self, department, course_number, section, new_times):
        course_key = (department, course_number, section)
        if course_key not in self.courses:
            raise Exception(f"Course {department} {course_number} (section {section}) not found.")
        course = self.courses[course_key]

        for student in course.students:
            student_schedule = student.schedule
            for current_course in student_schedule.courses:
                if current_course != course and Course.check_time_conflicts(current_course.times, new_times):
                    raise Exception(f"Time conflict for student {student.firstname} {student.lastname} in course {current_course.department} {current_course.course_number} (section {current_course.section})")

        if course.instructor is not None:
            instructor_schedule = course.instructor.schedule
            for current_course in instructor_schedule.courses:
                if current_course != course and Course.check_time_conflicts(current_course.times, new_times):
                    raise Exception(f"Time conflict for instructor {course.instructor.firstname} {course.instructor.lastname} in course {current_course.department} {current_course.course_number} (section {current_course.section})")

        course.change_time(new_times)